# Keşifsel Veri Analizi (EDA)

Bu çalışmanın amacı veri setinin yapısını incelemek, veri kalitesi problemlerini tespit etmek ve veri temizleme kararları almaktır.

In [4]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)

## Veri Setinin Boyutunun İncelenmesi

Bu adımda veri setindeki toplam satır ve sütun sayısı incelenmiştir. Veri boyutunun bilinmesi, veri setinin büyüklüğünü anlamak ve yapılacak işlemlerin kapsamını belirlemek açısından önemlidir.

In [5]:
# Dosya adını/path'ini kendi dosyana göre düzenle
df_raw = pd.read_csv("../data/raw/OnlineRetail.csv", encoding="latin1")

# Çalışma kopyası (ham veri hep dursun)
df = df_raw.copy()

print("Ham veri boyutu (satır, sütun):", df.shape)
df.head()

Ham veri boyutu (satır, sütun): (541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## Veri setinin genel yapısının incelenmesi

Bu aşamada veri tipleri ve eksik değerler kontrol edilmiştir.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## İstatistiksel Özet Bilgilerin İncelenmesi

Sayısal değişkenlere ait ortalama, minimum, maksimum ve standart sapma gibi özet istatistikler incelenmiştir. Bu adım veri dağılımı hakkında genel bir fikir edinmek için yapılmıştır.

In [7]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
InvoiceNo,541909,25900,573585,1114,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StockCode,541909,4070,85123A,2313,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Description,540455,4223,WHITE HANGING HEART T-LIGHT HOLDER,2369,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quantity,541909.0,NaN,NaN,NaN,9.55225,218.081158,-80995.0,1.0,3.0,10.0,80995.0
InvoiceDate,541909,23260,10/31/2011 14:41,1114,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UnitPrice,541909.0,NaN,NaN,NaN,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,NaN,NaN,NaN,15287.69057,1713.600303,12346.0,13953.0,15152.0,16791.0,18287.0
Country,541909,38,United Kingdom,495478,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Eksik değer analizi

Veri setindeki eksik değerlerin hangi sütunlarda bulunduğu incelenmiştir.

In [8]:
missing = df.isnull().sum().sort_values(ascending=False)
missing[missing > 0]

CustomerID     135080
Description      1454
dtype: int64

In [9]:
missing_rate = (df.isnull().mean() * 100).sort_values(ascending=False)
missing_rate[missing_rate > 0]

CustomerID     24.926694
Description     0.268311
dtype: float64

## Tekrarlanan Kayıtların Kontrolü

Veri setinde tekrar eden satırların olup olmadığı kontrol edilmiştir. Tekrarlanan kayıtlar analiz sonuçlarını yanıltabileceği için tespit edilmiştir.

In [10]:
dup_count = df.duplicated().sum()
print("Duplicate satır sayısı:", dup_count)

Duplicate satır sayısı: 5268


In [11]:
df[df.duplicated()].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,12/1/2010 11:45,1.25,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,12/1/2010 11:45,2.10,17908.0,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,12/1/2010 11:45,2.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,12/1/2010 11:45,4.95,17908.0,United Kingdom
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,12/1/2010 11:49,2.95,17920.0,United Kingdom


## Quantity Değerlerinin İncelenmesi

"Quantity" değişkeni mantıksal tutarlılık açısından incelenmiştir. Negatif değerlerin ürün iadesi veya sipariş iptali gibi işlemleri temsil edebileceği değerlendirilmiştir. Ayrıca quantity değeri sıfır olan kayıtlar kontrol edilmiştir.

In [12]:
neg_qty_count = (df["Quantity"] < 0).sum()
zero_qty_count = (df["Quantity"] == 0).sum()

print("Negatif Quantity satır sayısı:", neg_qty_count)
print("0 Quantity satır sayısı:", zero_qty_count)

# Negatif örnekler
df[df["Quantity"] < 0].head()

Negatif Quantity satır sayısı: 10624
0 Quantity satır sayısı: 0


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom


## UnitPrice Değerlerinin İncelenmesi

"UnitPrice" değişkeninde sıfır fiyatlı işlemler kontrol edilmiştir. Sıfır fiyatlı işlemler sistem düzeltmesi, promosyon veya veri giriş hatası olabileceği için incelenmiştir.

In [13]:
zero_price_count = (df["UnitPrice"] == 0).sum()
print("UnitPrice = 0 satır sayısı:", zero_price_count)

df[df["UnitPrice"] == 0].head()

UnitPrice = 0 satır sayısı: 2515


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom


In [15]:
df[df["UnitPrice"] == 0]["Description"].value_counts().head(10)

Description
check                            159
?                                 47
damages                           45
damaged                           43
found                             25
sold as set on dotcom             20
adjustment                        16
Damaged                           14
Unsaleable, destroyed.             9
FRENCH BLUE METAL DOOR SIGN 1      9
Name: count, dtype: int64

In [16]:
# 1️⃣ InvoiceDate'i datetime yap (EN ÖNEMLİ ADIM)
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

# kontrol
print("df InvoiceDate type:", df["InvoiceDate"].dtype)

df InvoiceDate type: datetime64[ns]


### Satış ve İade İşlemlerinin Ayrılması

Negatif quantity değerleri ürün iadesi veya sipariş iptali işlemlerini temsil ettiği için satış işlemlerinden ayrılmıştır.

In [17]:
sales_df = df[df["Quantity"] > 0].copy()     # satışlar
returns_df = df[df["Quantity"] < 0].copy()   # iadeler

print("sales_df shape:", sales_df.shape)
print("returns_df shape:", returns_df.shape)

print("sales_df InvoiceDate:", sales_df["InvoiceDate"].dtype)
print("returns_df InvoiceDate:", returns_df["InvoiceDate"].dtype)



sales_df shape: (531285, 8)
returns_df shape: (10624, 8)
sales_df InvoiceDate: datetime64[ns]
returns_df InvoiceDate: datetime64[ns]


In [18]:
sales_df["Year"] = sales_df["InvoiceDate"].dt.year
sales_df["Month"] = sales_df["InvoiceDate"].dt.month
sales_df["Day"] = sales_df["InvoiceDate"].dt.day
sales_df["Hour"] = sales_df["InvoiceDate"].dt.hour


In [19]:
sales_clean = sales_df.copy()

### Tekrarlanan Kayıtların Kaldırılması

Veri setinde tekrar eden satırlar analiz sonuçlarını yanıltabileceği için veri setinden çıkarılmıştır.

In [21]:
print("Silmeden önce:", df.shape)

df = df.drop_duplicates()

print("Sildikten sonra:", df.shape)

Silmeden önce: (541909, 8)
Sildikten sonra: (536641, 8)


### Sıfır Fiyatlı İşlemlerin Kaldırılması

Sıfır fiyatlı işlemler gerçek satış işlemlerini temsil etmediği için veri setinden çıkarılmıştır.

In [22]:
sales_df = sales_df[sales_df["UnitPrice"] > 0]

### CustomerID Eksik Kayıtların Kaldırılması

Müşteri bazlı analiz yapılabilmesi için CustomerID değeri eksik olan kayıtlar veri setinden çıkarılmıştır.

In [23]:
sales_df = sales_df.dropna(subset=["CustomerID"])

In [24]:
sales_df = sales_df.dropna(subset=["Description"])

# Her işlem için müşteri tarafından ödenen toplam tutarı hesaplamak amacıyla Quantity ve UnitPrice değişkenleri kullanılarak TotalPrice değişkeni oluşturulmuştur.

In [25]:
sales_df["TotalPrice"] = sales_df["Quantity"] * sales_df["UnitPrice"]

In [32]:
sales_df.info()
sales_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 397884 entries, 0 to 541908
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  float64       
 7   Country      397884 non-null  object        
 8   Year         397884 non-null  int32         
 9   Month        397884 non-null  int32         
 10  Day          397884 non-null  int32         
 11  Hour         397884 non-null  int32         
 12  TotalPrice   397884 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int32(4), int64(1), object(4)
memory usage: 36.4+ MB


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
Year           0
Month          0
Day            0
Hour           0
TotalPrice     0
dtype: int64

In [34]:
sales_df.to_csv("../data/processed/clean_data.csv", index=False)

## Keşifsel Veri Analizi Sonuçları (EDA Sonuç Özeti)

Bu aşamada veri seti detaylı şekilde incelenmiş ve analiz için uygun hale getirilmiştir. Keşifsel veri analizi sürecinde aşağıdaki işlemler gerçekleştirilmiştir:

### Yapılan İşlemler

- Veri setinin genel yapısı incelenmiştir.
- Veri tipleri kontrol edilmiştir.
- Eksik değerler tespit edilmiş ve gerekli temizleme işlemleri uygulanmıştır.
- Tekrarlanan kayıtlar veri setinden çıkarılmıştır.
- Negatif quantity değerleri iade işlemleri olarak ayrılmıştır.
- Sıfır fiyatlı işlemler veri setinden kaldırılmıştır.
- CustomerID ve Description değeri eksik olan kayıtlar temizlenmiştir.
- "InvoiceDate" değişkeni datetime formatına dönüştürülmüştür.
- Zaman bazlı analiz için Year, Month, Day ve Hour değişkenleri oluşturulmuştur.
- Her işlem için toplam harcamayı gösteren "TotalPrice" değişkeni oluşturulmuştur.

### Sonuç

Veri seti temizlenmiş, tutarlı hale getirilmiş ve makine öğrenmesi modelleri ile analiz için uygun duruma getirilmiştir. Bir sonraki aşamada müşteri bazlı analiz ve modelleme çalışmaları gerçekleştirilecektir.